In [ ]:
import numpy as np
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
shp = gpd.read_file("./data/dissolved/0812_dis.shp")
rst = rio.open('./data/2017/2017.08.15.nir.tif')
geo = shp.loc[90].geometry
geo

In [ ]:
def transform_pix(img, nodata=255, border_pix=2000):
    m = np.copy(img)
    row = m.shape[1]
    col = m.shape[2]
    for y in range(0, col):
        for x in range(0, row):
            if m[0, x, y] != nodata:
                # border
                if x == 0 or y == 0 or x == row-1 or y == col-1: 
                    m[0, x, y] = border_pix
                # top
                if x > 0 and m[0, x-1, y] == nodata: 
                    m[0, x, y] = border_pix
                # bottom
                if x < row-1 and m[0, x+1, y] == nodata:
                    m[0, x, y] = border_pix
                # left
                if y > 0 and m[0, x, y-1] == nodata:
                    m[0, x, y] = border_pix
                # right
                if y < col-1 and m[0, x, y+1] == nodata:
                    m[0, x, y] = border_pix
    return m

In [ ]:
# Mask `raster` image using polygon from shapefile
img, transform = mask(rst, [geo], crop=True, nodata=255)
img

In [ ]:
cimg = transform_pix(img)
print(cimg)
show(cimg)

In [ ]:
cp = np.ones(rst.read(1).shape, dtype=np.int16)
for x in shp.itertuples():
    crp_img, crp_trans = mask(rst, [x.geometry], crop=True, nodata=255)
    t_img = transform_pix(crp_img)
    t_img = t_img.reshape((t_img.shape[1], t_img.shape[2]))
    it = np.nditer(t_img, flags=["multi_index"])
    while not it.finished:
        j, i = it.multi_index
        y, x = ~rst.transform * crp_trans * (i, j)
        if it[0] == 2000:
            cp[int(x), int(y)] = 0 #it[0]
        it.iternext()

In [ ]:
profile = rst.profile
with rio.open('./data/truth.tif', 'w', **profile) as dst:
    dst.write(cp, 1)